<a href="https://colab.research.google.com/github/Joshua-Hill-Science/Landscapes/blob/ross/Project_4_Ross.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! unzip sample_data/archive.zip &> /dev/null

In [1]:
import tensorflow as tf
import pandas as pd
from tensorflow import keras
from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
import numpy as np
import os
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('fivethirtyeight')

from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import matplotlib.pyplot as plt
from tensorflow.keras.layers import MaxPooling2D
from keras import regularizers
from tensorflow.keras.layers import Dropout

**Reading in image data and scaling it**

In [ ]:
train_data_dir = 'seg_train/seg_train'
test_data_dir = 'seg_test/seg_test'

test_generator = ImageDataGenerator(rescale = 1.0/255).flow_from_directory(
        test_data_dir, 
        target_size=(64, 64), batch_size=64)

train_generator = ImageDataGenerator(rescale = 1.0/255).flow_from_directory(
        train_data_dir, 
        target_size=(64, 64), batch_size=64)

**Baseline Model**

In [ ]:
from tensorflow.keras.layers import MaxPooling2D

model = Sequential() 
model.add(Conv2D(64, (3, 3), activation='relu', padding='same', input_shape=(64, 64, 3),))# use_bias=True)) 
model.add(MaxPooling2D((2, 2))) 
model.add(Flatten()) 
model.add(Dense(128, activation='relu',)) #use_bias=True)) 
model.add(Dense(6, activation='sigmoid',))# use_bias=True))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
history = model.fit_generator(
        train_generator,
        epochs=10,
        validation_data = test_generator)

**Graphing**

In [ ]:
def history_graph(history):
    fig, (ax1, ax2) = plt.subplots(1,2,figsize=(16, 6))
    metrics_loss = ['loss','val_loss']
    for metric_loss in metrics_loss:
        ax1.plot(history.history[metric_loss], label=metric_loss)
        ax1.legend()
        ax1.set_title(f"Model's Loss per Epoch")
    metrics_acc = ['accuracy','val_accuracy']
    for metric_acc in metrics_acc:
        ax2.plot(history.history[metric_acc], label=metric_acc)
        ax2.legend()
        ax2.set_title(f"Model's Accuracy per Epoch")
    plt.tight_layout()

In [ ]:
history_graph(history)

**Tuning**

In [ ]:
from keras.callbacks import EarlyStopping, ModelCheckpoint
early_stopping = [EarlyStopping(monitor='val_loss', patience=4), 
                  ModelCheckpoint(filepath='best_model.h5', monitor='val_loss', save_best_only=True)]

In [ ]:
model = Sequential() 
model.add(Conv2D(64, (3, 3), activation='relu', padding='same', input_shape=(64, 64, 3),))# use_bias=True)) 
model.add(MaxPooling2D((2, 2))) 
model.add(Flatten()) 
model.add(Dense(128, activation='relu',)) #use_bias=True)) 
model.add(Dense(6, activation='sigmoid',))# use_bias=True))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
history2 = model.fit_generator(
        train_generator,
        epochs=10,
        callbacks = early_stopping,
        validation_data = test_generator)

In [ ]:
from keras.models import load_model
saved_model = load_model('best_model.h5')

In [ ]:
results_train = saved_model.evaluate_generator(train_generator, 400)
print(results_train)

print('----------')

results_test = saved_model.evaluate_generator(test_generator, 400)
print(results_test)

**L2 Regularization**

In [ ]:
l2_reg = Sequential() 
l2_reg.add(Conv2D(64, (3, 3), activation='relu', padding='same',kernel_regularizer=regularizers.l2(0.005), input_shape=(64, 64, 3),))
l2_reg.add(MaxPooling2D((2, 2))) 
l2_reg.add(Flatten()) 
l2_reg.add(Dense(128,kernel_regularizer=regularizers.l2(0.005), activation='relu',))
l2_reg.add(Dense(6, activation='sigmoid',))

l2_reg.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
l2_reg_hist = l2_reg.fit_generator(
        train_generator,
        epochs=30,
        validation_data = test_generator)

In [ ]:
def history_graph(history):
    fig, (ax1, ax2) = plt.subplots(1,2,figsize=(16, 6))
    metrics_loss = ['loss','val_loss']
    for metric_loss in metrics_loss:
        ax1.plot(history.history[metric_loss], label=metric_loss)
        ax1.legend()
        ax1.set_title(f"Model's Loss per Epoch")
    metrics_acc = ['accuracy','val_accuracy']
    for metric_acc in metrics_acc:
        ax2.plot(history.history[metric_acc], label=metric_acc)
        ax2.legend()
        ax2.set_title(f"Model's Accuracy per Epoch")
    plt.tight_layout()

In [ ]:
history_graph(l2_reg_hist)

**l1 Regularization**

In [ ]:
l1_reg = Sequential() 
l1_reg.add(Conv2D(64, (3, 3), activation='relu', padding='same',kernel_regularizer=regularizers.l1(0.005), input_shape=(64, 64, 3),))
l1_reg.add(MaxPooling2D((2, 2))) 
l1_reg.add(Flatten()) 
l1_reg.add(Dense(128,kernel_regularizer=regularizers.l1(0.005), activation='relu',))
l1_reg.add(Dense(6, activation='sigmoid',))

l1_reg.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
l1_reg_hist = l2_reg.fit_generator(
        train_generator,
        epochs=10,
        callbacks = early_stopping,
        validation_data = test_generator)

**Dropout**

In [ ]:
drop = Sequential() 
drop.add(Conv2D(64, (3, 3), activation='relu', padding='same',kernel_regularizer=regularizers.l1(0.005), input_shape=(64, 64, 3),))
drop.add(MaxPooling2D((2, 2))) 
drop.add(Dropout(0.25))
drop.add(Flatten()) 
drop.add(Dense(128,kernel_regularizer=regularizers.l2(0.005), activation='relu',))
drop.add(Dropout(0.25))
drop.add(Dense(6, activation='sigmoid',))
drop.add(Dropout(0.25))

drop.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
drop_hist = l2_reg.fit_generator(
        train_generator,
        epochs=10,
        callbacks = early_stopping,
        validation_data = test_generator)

**Dropout and L2**

In [ ]:
all = Sequential() 
all.add(Conv2D(64, (3, 3), activation='relu', padding='same',kernel_regularizer=regularizers.l2(0.002), input_shape=(64, 64, 3),))
all.add(MaxPooling2D((2, 2))) 
all.add(Flatten())
all.add(Dropout(0.25)) 
all.add(Dense(128,kernel_regularizer=regularizers.l2(0.004), activation='relu',))
all.add(Dropout(0.25))
all.add(Dense(64,kernel_regularizer=regularizers.l2(0.004), activation='relu',))
all.add(Dropout(0.25))
all.add(Dense(6, activation='sigmoid'))


all.compile(loss='categorical_crossentropy',
              optimizer='nadam',
              metrics=['accuracy'])

In [ ]:
all_hist = all.fit_generator(
        train_generator,
        epochs=30,
        validation_data = test_generator)
        #callbacks = early_stopping,

In [ ]:
history_graph(all_hist)

**Adding another L2 layer to help the fluctuation**

In [ ]:
l5_reg = Sequential() 
l5_reg.add(Conv2D(64, (3, 3), activation='relu', padding='same',kernel_regularizer=regularizers.l2(0.005), input_shape=(64, 64, 3),))
l5_reg.add(MaxPooling2D((2, 2))) 
l5_reg.add(Flatten()) 
l5_reg.add(Dense(128,kernel_regularizer=regularizers.l2(0.005), activation='relu'))
l5_reg.add(Dense(64,kernel_regularizer=regularizers.l2(0.005), activation='relu'))
l5_reg.add(Dense(6, activation='sigmoid'))

l5_reg.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
history_5 = l5_reg.fit_generator(
        train_generator,
        epochs=10,
        validation_data = test_generator)

**Final Model**

In [ ]:
from keras.optimizers import Adam
final = Sequential() 
final.add(Conv2D(64, (3, 3), activation='relu', padding='same',kernel_regularizer=regularizers.l2(0.003), input_shape=(64, 64, 3),))
final.add(MaxPooling2D((2, 2))) 
final.add(Flatten())
final.add(Dropout(0.30)) 
final.add(Dense(128,kernel_regularizer=regularizers.l2(0.003), activation='relu',))
final.add(Dropout(0.30))
final.add(Dense(6, activation='sigmoid'))
#default= 0.001
opt = tf.keras.optimizers.Adam(learning_rate=0.0005)
final.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

In [ ]:
checkpoint = tf.keras.callbacks.ModelCheckpoint("best_model.h5", save_best_only=True)
final_model = final.fit_generator(
        train_generator,
        epochs=22,
        validation_data = test_generator,
        callbacks=[checkpoint])

In [ ]:
model = tensorflow.keras.models.load_model("best_model.h5")

**Prediction**

In [ ]:
from PIL import Image

In [ ]:
im = Image.open("seg_pred/seg_pred/10012.jpg")

im

In [ ]:
image = tf.keras.preprocessing.image.load_img('seg_pred/seg_pred/10012.jpg', target_size=((64,64)))
input_arr = tf.keras.preprocessing.image.img_to_array(image)
input_arr = np.array([input_arr])
predictions = final_model.predict(input_arr)
predictions